# Training simplistic models
The exact conditions for protein crystallization include pH, Temperature, protein concentration, physical setup and buffer ingredients and concentrations. 
A proper combination of all of the above will result in crystallization, but the space of possibilities for conditions is immense.
As a POC, we will train a simple model to predict crystallization conditions separately.  

Important: (This is done for POC, While the biochemical meaning of predicting temperature or pH by themselves without the buffer is very limited.)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

VEC_COLS = list(np.arange(1280))
ENTRY_COLS = ['pdb_id', 'struct_method', 'crystal_method', 'ph', 'temp', 'pdbx_details', 'deposit_date', 'revision_date']


In [2]:
#load the pdb data
df = pd.read_csv('../data/pdb_data/processed_data.csv')
df.shape

#Load the ESM vectors were generated using crystoper/vectorize.py
vec_path = '../vectors/sequences/esm2.npy'
vectors = pd.DataFrame(np.load(vec_path))

In [5]:
#concat vectors to df
df = pd.concat([df, vectors], axis=1)
del vectors

df_vecs = df[['pdb_id'] + VEC_COLS]
df = df[ENTRY_COLS]

In [6]:
#count entities, avarage vectors and create a new df
avg = df_vecs.groupby('pdb_id')[VEC_COLS].sum().reset_index()

counts = df_vecs[['pdb_id']].groupby('pdb_id').size().reset_index()
counts.columns = ['pdb_id', 'n_entities']

df = df[ENTRY_COLS].drop_duplicates().merge(counts, how='left').merge(avg, how='left')

del avg
del counts


 # pH predicting model    

In [7]:
m = df.ph.notna()

print(f'Using {m.sum()} not-Nan pH records out of total {len(df)} records ({100*m.sum() / len(df):.2f}%) ' )

Using 154949 not-Nan pH records out of total 169048 records (91.66%) 


In [8]:
X = df[VEC_COLS][m]
y = df.ph[m]

TEST_SIZE = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [9]:
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"R² Score: {r2}\n")

In [ ]:
# 5.1: Linear Regression
print("Linear Regression:")
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)
evaluate_model(y_test, y_pred)

# 5.2: Decision Tree Regressor
# print("Decision Tree Regressor:")
# dt_reg = DecisionTreeRegressor(random_state=42)
# dt_reg.fit(X_train, y_train)
# y_pred = dt_reg.predict(X_test)
# evaluate_model(y_test, y_pred)

# # 5.3: Random Forest Regressor
# print("Random Forest Regressor:")
# rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_reg.fit(X_train, y_train)
# y_pred = rf_reg.predict(X_test)
# evaluate_model(y_test, y_pred)

# 5.4: Gradient Boosting Regressor
print("Gradient Boosting Regressor:")
gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_reg.fit(X_train, y_train)
y_pred = gb_reg.predict(X_test)
evaluate_model(y_test, y_pred)

Linear Regression:
Mean Squared Error: 1.5783552094644542
Mean Absolute Error: 1.000605664363712
R² Score: 0.04429993479285299

Gradient Boosting Regressor:
